In [2]:
import json
def add(x, y):
    return json.dumps({"result" : x + y})

def subtract(x, y):
    return json.dumps({"result" : x - y})

def multiply(x, y):
    return json.dumps({"result" : x * y})

def divide(x, y):
    if y == 0:
        return "Error! Division by zero."
    return json.dumps({"result" : x / y})

def get_tools_definition(function_name, description):
    return {
            "type": "function",
            "function": {
                "name": function_name,
                "description": description,
                "parameters": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number",
                            "description": "The first number",
                        },
                        "y": {
                            "type": "number",
                            "description": "the second number",
                        },
                    },
                    "required": ["x", "y"],
                },
            },
        }

tools = [
    get_tools_definition("add", "add x to y"),
    get_tools_definition("subtract", "subtract y from x"),
    get_tools_definition("multiply", "x multiply y"),
    get_tools_definition("divide", "x divide y"),
]



In [3]:
from openai import OpenAI
import json

client = OpenAI()

messages = [{"role": "user", "content": "What does 1024 + 10086 equal to?"}]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)

In [4]:
response_message = response.choices[0].message
print(response_message.content)

None


In [9]:
tools_info = response_message.tool_calls
print(tools_info)

[ChatCompletionMessageToolCall(id='call_cGQpBmKgST5jZyh4JHXArKI5', function=Function(arguments='{"x":1024,"y":10086}', name='add'), type='function')]


In [10]:
tool_calls = response_message.tool_calls
available_functions = {
    "add": add,
    "subtract": subtract,
    "multiply": multiply,
    "divide": divide,
}

for tool_call in tool_calls:
    function_name = tool_call.function.name
    tool_call_id = tool_call.id
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)

    # 执行函数结果
    function_response = function_to_call(
        x=function_args.get("x"),
        y=function_args.get("y"),
    )
    print(function_response)
    # 从 tool_calls 中拿到需要调用的函数以及参数，并且实际调用对应的函数

{"result": 11110}


In [ ]:
messages.append(response_message)
messages.append(
    {
        "tool_call_id": tool_call_id,
        "role": "tool",
        "name": function_name,
        "content": str(function_response),
    }
)  

final_response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
)
print(final_response.choices[0].message.content)

In [8]:
messages.append(response_message)